# Shot‑Prompt Patterns Playground  
Explore **zero‑shot**, **one‑shot**, and **few‑shot** prompting in a single interface.

**Objective:**  
See how adding examples (shots) in the prompt changes an LLM’s output accuracy, tone, and depth.  
Works in Colab or any local Jupyter (just set `OPENAI_API_KEY`).

In [ ]:
!pip -q install openai ipywidgets

In [ ]:
import os, openai, json, ipywidgets as w
from IPython.display import display, Markdown

# 🔑  Enter your key (or leave blank if calling a local model wrapper)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'sk-')

task_box = w.Textarea(
    value="Translate to French:",
    placeholder="Describe your task here…",
    description="Task:",
    layout=w.Layout(width='100%', height='60px'))

shot_slider = w.IntSlider(value=0, min=0, max=5, step=1,
                          description='Shots (k):')

examples_box = w.Textarea(
    value="Example EN: Hello\nExample FR: Bonjour",
    placeholder="Provide k example pairs separated by newline\nFormat: input ⇢ output",
    description="Examples:",
    layout=w.Layout(width='100%', height='120px'))

temperature_slider = w.FloatSlider(value=0.7, min=0, max=1.5, step=0.1,
                                   description='Temperature')

run_btn = w.Button(description='Run 👉')
out = w.Output()

def build_messages():
    k = shot_slider.value
    messages = [{'role':'user','content': task_box.value}]
    if k > 0:
        lines = [l.strip() for l in examples_box.value.splitlines() if l.strip()]
        pair_lines = lines[:k]  # grab first k example lines
        for ln in pair_lines:
            if '⇢' in ln:
                src, tgt = ln.split('⇢', 1)
            elif ':' in ln:
                src, tgt = ln.split(':', 1)
            else:
                continue
            messages.append({'role':'user','content': src.strip()})
            messages.append({'role':'assistant','content': tgt.strip()})
    # final user prompt again
    messages.append({'role':'user','content': task_box.value})
    return messages

def run(b):
    with out:
        out.clear_output()
        messages = build_messages()
        print("Prompt messages:", json.dumps(messages, indent=2))
        try:
            resp = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                messages=messages,
                temperature=temperature_slider.value,
                max_tokens=256
            )
            display(Markdown("### ✨ Model Response\n" + resp.choices[0].message.content))
        except Exception as e:
            print("Error:", e)

run_btn.on_click(run)

ui = w.VBox([task_box, shot_slider, examples_box,
             temperature_slider, run_btn, out])
display(ui)
print("Adjust *Shots* and supply examples to compare outputs.")

---  
### Suggested Experiments  
1. Try `k=0` vs `k=3` on classification tasks.  
2. Observe how example **ordering** changes output.  
3. Combine with higher temperature for extra creativity.